In [1]:
import pandas as pd
import numpy as np
import os
import re

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Gapminder World Data set



## Source files

In [51]:
# indicator name and their hash names. The hashs are used as file name in 
# indicator json folder waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/indicators/
idt = pd.read_json('../waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/meta/quantities.json')

# groupings, there are 5 ways of grouping of countries in the data set.
gps = pd.read_json('../../ddf--gapminder_world/waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/meta/area_categorizarion.json')

# Country properities
geo = pd.read_excel('../waffle-server-importers-exporters-world-legacy-with-data/data/synonym/country_synonyms.xlsx')

# Region properities
reg_f = open('../waffle-server-importers-exporters-world-legacy-with-data/data/synonym/regions.json')
regs = json.load(reg_f)


## helper function

In [5]:
def to_concept_id(s, sub='[/ -\.\*";]+', sep='_'):
    '''convert a string to lowercase alphanumeric + underscore id for concepts'''
    
    if s is np.nan:
        return s
    
    s1 = re.sub(sub, sep, s.strip())
#     s1 = re.sub(r'\[.*\]', '', s1)
    s1 = s1.replace('\n', '')

    if s1[-1] == sep:  # remove the last underscore
        s1 = s1[:-1]

    return s1.lower()


In [60]:
def rename_col(s):
    
    real = idt[idt['-t-ind'] == s]['-t-name'].iloc[0]
    cid = concepts[concepts['name'] == real]['concept'].iloc[0]
#     print(real, ': ', cid)
    
    return cid

In [61]:
def rename_geo(s):
    return geo_.get_value(s, 'ISO3dig_ext')

# Entities of country gourps

In [62]:
regd = {}  # a dictionary, which keys are region id and values are region names

for i in regs:
    regd[i.get(list(i.keys())[0])] = list(i.keys())[0]

In [63]:
for i, n in gps.n.apply(to_concept_id).iteritems():
    df = pd.DataFrame([], columns=[n, 'name', n+'_id', 'is--'+n])
    df[n+'_id'] = gps.iloc[i]['groupings'].keys()
    if i == 4:
        df[n] = df[n+'_id'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+', sep=''))
    else:
        df[n] = df[n+'_id'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+'))

    df['name'] = df[n+'_id'].apply(lambda x: regd[x])
    df['is--'+n] = 'TRUE'

    df.to_csv('./output/ddf--entities--geo--'+n+'.csv', index=False)

# Entities of Country

In [65]:
geo_ = geo[['ISO3dig_ext', 'Gwid']]
geo_ = geo_.set_index('Gwid')
geo_2 = geo.set_index('Gwid').drop('ISO3dig_ext', axis=1)

In [93]:
country = geo_.copy()

for i, n in gps.n.apply(to_concept_id).iteritems():

    res = {}

    for k, v in gps.iloc[i]['groupings'].items():
    #     print(regd[k])
        for c in v:
            if c:
#                 if c == 'i144':
#                     print(k, regd[k])
                if i == 4:
                    res[c] = to_concept_id(regd[k], sub='[/ -\.\*";\[\]]+', sep='')
                else:
                    res[c] = to_concept_id(regd[k], sub='[/ -\.\*";\[\]]+')

    ser = pd.Series(res)
    
    country[n] = ser

i265 East Asia & Pacific
i269 Lower middle income
i270 Coastline
i272 G77
i274 Others
i276 [Asia]
i279 Christian


In [79]:
country2 = pd.concat([country, geo_2], axis=1)

In [80]:
country2 = country2.reset_index()
country2 = country2.rename(columns={'NAME': 'Upper Case Name', 'Use Name': 'Name', 'ISO3dig_ext': 'country'})
country2.columns = list(map(to_concept_id, country2.columns))
country2['is--country'] = 'TRUE'

In [81]:
ccs = country2.columns[8:]

In [82]:
c2c = ['country', 'gwid', 'name', 'geographic_regions', 'income_groups', 'landlocked',
       'g77_and_oecd_countries', 'geographic_regions_in_4_colors',
       'main_religion_2008', 'gapminder_list', 'alternative_1',
       'alternative_2', 'alternative_3', 'alternative_4_cdiac', 'pandg',
       'god_id', 'alt_5', 'upper_case_name', 'code', 'number', 'arb1', 'arb2',
       'arb3', 'arb4', 'arb5', 'arb6', 'is--country']

country2.loc[:, c2c].to_csv('./output/ddf--entities--geo--country.csv', index=False)

# Concepts

In [2]:
concepts = pd.read_csv('./source/graph_settings - Indicators.csv')

In [3]:
dsc = concepts.columns

In [6]:
concepts.columns = list(map(to_concept_id, concepts.columns))

In [7]:
concepts['concept_type'] = 'measure'

In [8]:
concepts = concepts.drop('download', axis=1)

In [9]:
concepts = concepts.iloc[:, [5, 0, 1,2,3,4,6,7]]

In [10]:
concepts = concepts.rename(columns={'ddf_id':'concept'})

In [11]:
concepts['concept_new'] = concepts['concept'].apply(to_concept_id)

In [13]:
concepts.head()

,concept,name,tooltip,menu_level1,menu_level_2,indicator_url,scale,concept_type,concept_new
0,children_per_woman_total_fertility,Children per woman (total fertility),Total fertility rate. The number of children t...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure,children_per_woman_total_fertility
1,co2_emissions_tonnes_per_person,CO2 emissions (tonnes per person),Carbon dioxide emissions from the burning of f...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure,co2_emissions_tonnes_per_person
2,income_per_person_gdppercapita_ppp_inflation_a...,"Income per person (GDP/capita, PPP$ inflation-...",Gross domestic product per person adjusted for...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,log,measure,income_per_person_gdppercapita_ppp_inflation_a...
3,child_mortality_0_5_year_olds_dying_per_1000_born,"Child mortality (0-5 year-olds dying per 1,000...",Death of children under five years of age per ...,NaN,NaN,http://spreadsheets.google.com/pub?key=0ArfEDs...,log,measure,child_mortality_0_5_year_olds_dying_per_1000_born
4,life_expectancy_years,Life expectancy (years),The average number of years a newborn child wo...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,measure,life_expectancy_years


In [90]:
dsc.drop('ddf_id')

Index(['Name', 'Tooltip', 'Menu level1', 'Menu level 2', 'Indicator url',
       'Download', 'Scale'],
      dtype='object')

In [35]:
len(concepts)

544

In [14]:
cc = concepts[['concept', 'name', 'concept_type', 'tooltip', 'indicator_url']].copy()

In [15]:
cc.columns = [['concept', 'name', 'concept_type', 'tooltip', 'indicator_url']]

In [16]:
k = concepts[concepts.concept == u'———————————————————————'].index

In [17]:
cc = cc.drop(k)

In [18]:
cc['drill_up'] = np.nan
cc['domain'] = np.nan

In [20]:
dc = pd.DataFrame([], columns=cc.columns)

In [21]:
dcl = list(map(to_concept_id, gps.n.values))

In [42]:
ccs_id = list(map(to_concept_id, ccs))

In [43]:
dcl_ = np.r_[dcl, ['geo', 'country','time', 'name', 'gwid'], [x+'_id' for x in dcl], ccs_id, ['tooltip', 'indicator_url']]
dcl_2 = np.r_[gps.n.values, ['Geo', 'Country','Time', 'Name', 'Gwid'], [x+' id' for x in gps.n.values], ccs, ['tooltip', 'indicator_url']]

In [44]:
dcl_

array(['geographic_regions', 'income_groups', 'landlocked',
       'g77_and_oecd_countries', 'geographic_regions_in_4_colors',
       'main_religion_2008', 'geo', 'country', 'time', 'name', 'gwid',
       'geographic_regions_id', 'income_groups_id', 'landlocked_id',
       'g77_and_oecd_countries_id', 'geographic_regions_in_4_colors_id',
       'main_religion_2008_id', 'gapminder_list', 'alternative_1',
       'alternative_2', 'alternative_3', 'alternative_4_cdiac', 'pandg',
       'god_id', 'alt_5', 'upper_case_name', 'code', 'number', 'arb1',
       'arb2', 'arb3', 'arb4', 'arb5', 'arb6', 'tooltip', 'indicator_url'], 
      dtype='<U33')

In [45]:
dc['concept'] = dcl_
dc['name'] = dcl_2

In [46]:
dc['concept_type'] = 'string'
dc.loc[:5, 'concept_type'] = 'entity_set'
dc.loc[6, 'concept_type'] = 'entity_domain'
dc.loc[7, 'concept_type'] = 'entity_set'
dc.loc[8, 'concept_type'] = 'time'

In [47]:
# dc['concept_type'] = 'string'
dc.loc[:5, 'domain'] = 'geo'

dc.loc[7, 'drill_up'] = dcl
dc.loc[7, 'domain'] = 'geo'

In [48]:
dc

,concept,name,concept_type,tooltip,indicator_url,drill_up,domain
0,geographic_regions,Geographic regions,entity_set,NaN,NaN,NaN,geo
1,income_groups,Income groups,entity_set,NaN,NaN,NaN,geo
2,landlocked,Landlocked,entity_set,NaN,NaN,NaN,geo
3,g77_and_oecd_countries,G77 and OECD countries,entity_set,NaN,NaN,NaN,geo
4,geographic_regions_in_4_colors,Geographic regions in 4 colors,entity_set,NaN,NaN,NaN,geo
5,main_religion_2008,Main religion 2008,entity_set,NaN,NaN,NaN,geo
6,geo,Geo,entity_domain,NaN,NaN,NaN,NaN
7,country,Country,entity_set,NaN,NaN,"[geographic_regions, income_groups, landlocked...",geo
8,time,Time,time,NaN,NaN,NaN,NaN
9,name,Name,string,NaN,NaN,NaN,NaN


In [49]:
c_all = pd.concat([dc, cc])

In [50]:
c_all.to_csv('./output/ddf--concepts.csv', index=False)

# data points

In [56]:
path = '../waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/indicators/'

In [57]:
fs = os.listdir(path)

In [95]:
p1 = os.path.join(path, fs[300])
d1 = pd.read_json(p1)

In [96]:
d1.geo = d1.geo.apply(rename_geo)

In [54]:
idt[idt['-t-ind'] == '0AgogXXPMARyldC1rcTI5OU50Mnc1djdkNXpnWUFrZmc']['-t-name'].iloc[0]

'Murdered children 0-14 (per 100,000 people)'

In [115]:
rename_col('rCyfwvThkbHVlNVw48vHybg')

'males_aged_15_24_employment_rate_percent'

In [ ]:
# now try to make data points

In [59]:
for f in fs[1:]:
    p = os.path.join(path, f)
    
    col = f[:-5]
    col_r = rename_col(col)
  
    d = pd.read_json(p)

    if 'geo' in d.columns:
        d['geo'] = d['geo'].apply(rename_geo)
        d = d.rename(columns={col:col_r}) #ok?
        d.to_csv('./output/ddf--datapoints--'+col_r+'--by--geo--time.csv', index=False)
    else:
        print(col, ':', col_r)
    
    

rj8WsGGNXkijP447qnY5RLg : ———————————————————————
rMF2H57_Vd8TdobvtNhYuwg : ———————————————————————
tAwlNo30o6vVgDUk0EmDolQ : ———————————————————————
tDkfgIt5oQ_ZJQ7IBmUvsUg : ———————————————————————
tdKPTXS1U5Jn3qAQICk4-Vw : ———————————————————————
teiT4rPfnWcd5mCldIm4Zhw : ———————————————————————
tFc2tyrcWLcUyGhq6mTAbDg : ———————————————————————
tFrUv6vnhhO4Jr1czdG_p0Q : ———————————————————————
tIL264Nlw0AlIWlXuCWWNmQ : ———————————————————————
tJwP0x54AZ-X-bGL-RebrpA : ———————————————————————
tLiJLI_rimqrdQRWdamPPRQ : ———————————————————————
tnDimTzGwU_BbmtOMHOUFFQ : ———————————————————————
ts7TQEIg0amXovCDzQxV5EA : ———————————————————————
tYvQLeBTuoKsyofvwGqS-9w : ———————————————————————


In [102]:
fs[1][:-5]

'0AgogXXPMARyldC1rcTI5OU50Mnc1djdkNXpnWUFrZmc'

In [106]:
len(idt)

542

In [107]:
len(concepts)

544

---

# Other Todos

DO NOT try to run below script again. Because the en.json and meta.json have been changed and re-run the script will result at wrong outputs.

In [332]:
en = open('/Users/semio/src/work/Gapminder/vizabi/.data/translation/en.json')
enj = json.load(en)

In [323]:
idts = cc[['concept', 'name']].set_index('concept').to_dict()['name']

In [324]:
idts_new = {}

for k, v in idts.items():
    idts_new['indicator/'+k] = v

In [327]:
enj.update(idts_new)

In [333]:
enj   # problem in to_concept_id function: tsunami___affected_annual_number

{'buttons/about': 'About',
 'buttons/axes': 'X and Y',
 'buttons/axesmc': 'X and Y',
 'buttons/axis_x': 'X axis',
 'buttons/axis_y': 'Y axis',
 'buttons/color': 'Color',
 'buttons/colors': 'Color',
 'buttons/deselect': 'Deselect',
 'buttons/desktop': 'Desktop',
 'buttons/expand': 'Expand',
 'buttons/find': 'Find',
 'buttons/label': 'Labels',
 'buttons/lock': 'Lock',
 'buttons/more_options': 'Options',
 'buttons/ok': 'OK',
 'buttons/opacity': 'Opacity',
 'buttons/opacityNonselect': 'Opacity of non-selected',
 'buttons/opacityRegular': 'Regular opacity',
 'buttons/presentation': 'Present',
 'buttons/show': 'Show',
 'buttons/showall': 'Show all',
 'buttons/size': 'Size',
 'buttons/size_label': 'Size',
 'buttons/speed': 'Speed',
 'buttons/stack': 'Stack',
 'buttons/trails': 'Trails',
 'buttons/unexpand': 'Restore',
 'check/adaptMinMaxZoom': 'Follow bubbles with zoom',
 'check/presentation': 'Increase text size for presentation',
 'check/removeLabelBox': 'Remove label boxes',
 'hints/bubbl/

In [331]:
en = open('/Users/semio/src/work/Gapminder/vizabi/.data/translation/en.json', 'w')
json.dump(enj, en, indent=1, sort_keys=True)

In [341]:
indb = concepts[['concept_id', 'name', 'scale']].drop(k)

In [343]:
indb = indb.set_index('concept_id')
indb['use'] = 'indicator'

In [226]:
def scal(s):
    if s is not np.nan:
        if s == 'lin':
            return ['linear']
        else:
            return [s]
    else:
        return s

In [545]:
indb['scales'] = indb['scale'].apply(scal)

In [359]:
indb.drop('scale', axis=1).to_dict('index')

{'20120905_extreme_poverty_percent_people_below_125_a_day': {'name': 'Extreme poverty (% people below $1.25 a day) version 20120905',
  'scales': ['lin'],
  'use': 'indicator'},
 'adults_with_hiv_percent_age_15_49': {'name': 'Adults with HIV (%, age 15-49)',
  'scales': ['log'],
  'use': 'indicator'},
 'age_at_1st_marriage_women': {'name': 'Age at 1st marriage (women)',
  'scales': ['lin'],
  'use': 'indicator'},
 'aged_15_24_employment_rate_percent': {'name': 'Aged 15-24 employment rate (%)',
  'scales': ['lin'],
  'use': 'indicator'},
 'aged_15_24_unemployment_rate_percent': {'name': 'Aged 15-24 unemployment rate (%)',
  'scales': ['lin'],
  'use': 'indicator'},
 'aged_15_64_labour_force_participation_rate_percent': {'name': 'Aged 15-64 labour force participation rate (%)',
  'scales': ['lin'],
  'use': 'indicator'},
 'aged_15plus_employment_rate_percent': {'name': 'Aged 15+ employment rate (%)',
  'scales': ['lin'],
  'use': 'indicator'},
 'aged_15plus_labour_force_participation_rat

In [589]:
indb_0 = json.load(open('/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata.json'))

In [590]:
indb_0['indicatorsDB'].update(to_dict_dropna(indb.drop('scale', axis=1)))

In [591]:
indb_0['indicatorsDB']

{'20120905_extreme_poverty_percent_people_below_125_a_day': {'name': 'Extreme poverty (% people below $1.25 a day) version 20120905',
  'scales': ['linear'],
  'use': 'indicator'},
 '_default': {'scales': ['ordinal'], 'sourceLink': '', 'use': 'constant'},
 'adults_with_hiv_percent_age_15_49': {'name': 'Adults with HIV (%, age 15-49)',
  'scales': ['log'],
  'use': 'indicator'},
 'age': {'scales': ['linear', 'log'], 'sourceLink': '', 'use': 'indicator'},
 'age_at_1st_marriage_women': {'name': 'Age at 1st marriage (women)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_24_employment_rate_percent': {'name': 'Aged 15-24 employment rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_24_unemployment_rate_percent': {'name': 'Aged 15-24 unemployment rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_64_labour_force_participation_rate_percent': {'name': 'Aged 15-64 labour force participation rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 '

In [225]:
# try to remove NaNs
from pandas import compat 

def to_dict_dropna(data):
  return dict((k, v.dropna().to_dict()) for k, v in data.iterrows())

In [653]:
to_dict_dropna(indb.drop('scale', axis=1))

{'20120905_extreme_poverty_percent_people_below_125_a_day': {'name': 'Extreme poverty (% people below $1.25 a day) version 20120905',
  'scales': ['linear'],
  'use': 'indicator'},
 'adults_with_hiv_percent_age_15_49': {'name': 'Adults with HIV (%, age 15-49)',
  'scales': ['log'],
  'use': 'indicator'},
 'age_at_1st_marriage_women': {'name': 'Age at 1st marriage (women)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_24_employment_rate_percent': {'name': 'Aged 15-24 employment rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_24_unemployment_rate_percent': {'name': 'Aged 15-24 unemployment rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_64_labour_force_participation_rate_percent': {'name': 'Aged 15-64 labour force participation rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15plus_employment_rate_percent': {'name': 'Aged 15+ employment rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15plus_labour_force

In [377]:
intree = concepts[['concept_id', 'name', 'menu_level1', 'menu_level_2']].drop(k)

In [378]:
intree['ml1'] = intree['menu_level1'].apply(to_concept_id).fillna('_root')
intree['ml2'] = intree['menu_level_2'].apply(to_concept_id)

In [617]:
t = intree.set_index(['ml1', 'ml2'])['concept_id'].sort_index().copy()

In [634]:
list(t['economy'].iteritems())

[('aid_given', 'aid_given_2007_us'),
 ('aid_given', 'aid_given_per_person_2007_us'),
 ('aid_given', 'aid_given_percent_of_gni'),
 ('aid_given', 'water_and_sanitation_aid_given_percent_of_aid'),
 ('aid_given', 'economical_infrastructure_aid_given_percent_of_aid'),
 ('aid_given', 'production_sector_aid_given_percent_of_aid'),
 ('aid_given', 'multisector_cross_cutting_aid_given_percent_of_aid'),
 ('aid_given', 'education_aid_given_percent_of_aid'),
 ('aid_given', 'health_aid_given_percent_of_aid'),
 ('aid_given', 'population_policies_aid_given_percent_of_aid'),
 ('aid_given', 'government_and_society_aid_given_percent_of_aid'),
 ('aid_given', 'other_social_services_aid_given_percent_of_aid'),
 ('aid_received', 'aid_received_total_us_inflation_adjusted'),
 ('aid_received', 'aid_received_per_person_current_us'),
 ('aid_received', 'aid_received_percent_of_gni'),
 ('debt_trade', 'debt_to_foreigners_by_public_and_private_percent_of_gni'),
 ('debt_trade', 'total_reserves_percent_of_debt_to_forei

In [592]:
# adding _root level
tmp = [{'id': x} for x in t.loc['_root'].values]
indb_0['indicatorsTree']['children'] = [*indb_0['indicatorsTree']['children'], *tmp]

In [593]:
indb_0['indicatorsTree']['children']

[{'id': 'time'},
 {'children': [{'id': 'geo'}, {'id': 'geo.name'}, {'id': 'geo.region'}],
  'id': '_properties'},
 {'children': [{'children': [{'id': 'gini'}], 'id': 'inequality'},
   {'id': 'gdp_p_cap_const_ppp2011_dollar'},
   {'id': 'gdp_const_ppp2011_dollar'}],
  'id': 'economy'},
 {'children': [{'id': 'life_expectancy'},
   {'id': 'child_mortality_rate_per1000'},
   {'id': 'fertility_rate'}],
  'id': 'health'},
 {'id': 'population'},
 {'id': '_default'},
 {'id': 'children_per_woman_total_fertility'},
 {'id': 'co2_emissions_tonnes_per_person'},
 {'id': 'income_per_person_gdppercapita_ppp_inflation_adjusted'},
 {'id': 'child_mortality_0_5_year_olds_dying_per_1000_born'},
 {'id': 'life_expectancy_years'}]

In [426]:
t.groupby(level=0).groups.keys()

dict_keys(['economy', 'education', 'for_advanced_users', '_root', 'health', 'work', 'population', 'infrastructure', 'energy', 'environment', 'society'])

In [516]:
l0 = t.loc[t.groupby(level=0).groups['society']]

In [484]:
l0.index.unique()

array([('population', nan), ('population', 'both_sexes_by_age'),
       ('population', 'both_sexes_by_age_total'),
       ('population', 'female_by_age'), ('population', 'male_by_age'),
       ('population', 'population_growth'), ('population', 'sex_ratio'),
       ('population', 'urbanization')], dtype=object)

In [463]:
l0['health'].index.unique()

array([nan, 'cancer_female', 'cancer_male', 'causes_of_child_deaths',
       'causes_of_newborn_deaths', 'health_economics', 'hiv',
       'injuries_and_accidents', 'malaria', 'maternal_health',
       'mental_health', 'newborn_infants', 'nutrition', 'risk_factors',
       'tb_tuberculosis', 'teeth', 'vaccine'], dtype=object)

In [472]:
l0['health']

nan

In [630]:
tmp = []
j = 0
sss = 'economy'

l0 = t[sss]


for i, v in l0.iteritems():
    if i is np.nan:
        tmp.append({'id': v})


for i in l0.index.unique(): 
    if i is np.nan:
        continue
    else:
        tmp2 = {}
        tmp2['id'] = i
        tmp2['children'] = []
        
        if isinstance(l0[i], str):
            tmp2['children'].append({'id':l0[i]})
        else:

            for c in l0[i].values:
                tmp2['children'].append({'id': c})

            tmp.append(tmp2)
            


In [631]:
tmp

[{'children': [{'id': 'aid_given_2007_us'},
   {'id': 'aid_given_per_person_2007_us'},
   {'id': 'aid_given_percent_of_gni'},
   {'id': 'water_and_sanitation_aid_given_percent_of_aid'},
   {'id': 'economical_infrastructure_aid_given_percent_of_aid'},
   {'id': 'production_sector_aid_given_percent_of_aid'},
   {'id': 'multisector_cross_cutting_aid_given_percent_of_aid'},
   {'id': 'education_aid_given_percent_of_aid'},
   {'id': 'health_aid_given_percent_of_aid'},
   {'id': 'population_policies_aid_given_percent_of_aid'},
   {'id': 'government_and_society_aid_given_percent_of_aid'},
   {'id': 'other_social_services_aid_given_percent_of_aid'}],
  'id': 'aid_given'},
 {'children': [{'id': 'aid_received_total_us_inflation_adjusted'},
   {'id': 'aid_received_per_person_current_us'},
   {'id': 'aid_received_percent_of_gni'}],
  'id': 'aid_received'},
 {'children': [{'id': 'debt_to_foreigners_by_public_and_private_percent_of_gni'},
   {'id': 'total_reserves_percent_of_debt_to_foreigners'},
  

In [632]:
# adding to exists keys
indb_0['indicatorsTree']['children'][2]['children'] = [*indb_0['indicatorsTree']['children'][2]['children'], *tmp]

In [633]:
indb_0['indicatorsTree']['children'][2]['children']

[{'children': [{'id': 'gini'}], 'id': 'inequality'},
 {'id': 'gdp_p_cap_const_ppp2011_dollar'},
 {'id': 'gdp_const_ppp2011_dollar'},
 {'children': [{'id': 'aid_given_2007_us'},
   {'id': 'aid_given_per_person_2007_us'},
   {'id': 'aid_given_percent_of_gni'},
   {'id': 'water_and_sanitation_aid_given_percent_of_aid'},
   {'id': 'economical_infrastructure_aid_given_percent_of_aid'},
   {'id': 'production_sector_aid_given_percent_of_aid'},
   {'id': 'multisector_cross_cutting_aid_given_percent_of_aid'},
   {'id': 'education_aid_given_percent_of_aid'},
   {'id': 'health_aid_given_percent_of_aid'},
   {'id': 'population_policies_aid_given_percent_of_aid'},
   {'id': 'government_and_society_aid_given_percent_of_aid'},
   {'id': 'other_social_services_aid_given_percent_of_aid'}],
  'id': 'aid_given'},
 {'children': [{'id': 'aid_received_total_us_inflation_adjusted'},
   {'id': 'aid_received_per_person_current_us'},
   {'id': 'aid_received_percent_of_gni'}],
  'id': 'aid_received'},
 {'childre

In [628]:
indb_0['indicatorsTree']['children'][3]['children'] = [*indb_0['indicatorsTree']['children'][3]['children'], *tmp]

In [629]:
indb_0['indicatorsTree']['children'][3]

{'children': [{'id': 'life_expectancy'},
  {'id': 'child_mortality_rate_per1000'},
  {'id': 'fertility_rate'},
  {'children': [{'id': 'breast_cancer_new_cases_per_100000_women'},
    {'id': 'cervical_cancer_new_cases_per_100000_women'},
    {'id': 'colonandrectum_cancer_new_cases_per_100000_women'},
    {'id': 'liver_cancer_new_cases_per_100000_women'},
    {'id': 'lung_cancer_new_cases_per_100000_women'},
    {'id': 'stomach_cancer_new_cases_per_100000_women'},
    {'id': 'breast_cancer_number_of_new_female_cases'},
    {'id': 'cervical_cancer_number_of_new_female_cases'},
    {'id': 'colonandrectum_cancer_number_of_new_female_cases'},
    {'id': 'liver_cancer_number_of_new_female_cases'},
    {'id': 'lung_cancer_number_of_new_female_cases'},
    {'id': 'stomach_cancer_number_of_new_female_cases'},
    {'id': 'breast_cancer_deaths_per_100000_women'},
    {'id': 'cervical_cancer_deaths_per_100000_women'},
    {'id': 'colonandrectum_cancer_deaths_per_100000_women'},
    {'id': 'liver_ca

In [635]:
for ks in ['education', 'for_advanced_users', 'work', 'population', 'infrastructure', 'energy', 'environment', 'society']:

    tmp = []
    j = 0
    sss = ks

    l0 = t[sss]


    for i, v in l0.iteritems():
        if i is np.nan:
            tmp.append({'id': v})


    for i in l0.index.unique(): 
        if i is np.nan:
            continue
        else:
            tmp2 = {}
            tmp2['id'] = i
            tmp2['children'] = []

            if isinstance(l0[i], str):
                tmp2['children'].append({'id':l0[i]})
            else:

                for c in l0[i].values:
                    tmp2['children'].append({'id': c})

                tmp.append(tmp2)

    indb_0['indicatorsTree']['children'].append({'id':ks, 'children':tmp})

In [488]:
indb_0['indicatorsTree']['children'][4]['children'] = tmp

In [489]:
indb_0['indicatorsTree']['children'][4]

{'children': [{'id': 'children_and_elderly_per_100_adults'},
  {'children': [{'id': 'population_aged_0_4_years_both_sexes_percent'},
    {'id': 'population_aged_5_9_years_both_sexes_percent'},
    {'id': 'population_aged_10_14_years_both_sexes_percent'},
    {'id': 'population_aged_15_19_years_both_sexes_percent'},
    {'id': 'population_aged_20_39_years_both_sexes_percent'},
    {'id': 'population_aged_40_59_years_both_sexes_percent'},
    {'id': 'population_aged_60plus_years_both_sexes_percent'},
    {'id': 'population_aged_0_4_years_both_sexes_percent'},
    {'id': 'population_aged_5_9_years_both_sexes_percent'},
    {'id': 'population_aged_10_14_years_both_sexes_percent'},
    {'id': 'population_aged_15_19_years_both_sexes_percent'},
    {'id': 'population_aged_20_39_years_both_sexes_percent'},
    {'id': 'population_aged_40_59_years_both_sexes_percent'},
    {'id': 'population_aged_60plus_years_both_sexes_percent'},
    {'id': 'population_aged_0_4_years_both_sexes_percent'},
    {

In [637]:
indb_0

{'indicatorsDB': {'20120905_extreme_poverty_percent_people_below_125_a_day': {'name': 'Extreme poverty (% people below $1.25 a day) version 20120905',
   'scales': ['linear'],
   'use': 'indicator'},
  '_default': {'scales': ['ordinal'], 'sourceLink': '', 'use': 'constant'},
  'adults_with_hiv_percent_age_15_49': {'name': 'Adults with HIV (%, age 15-49)',
   'scales': ['log'],
   'use': 'indicator'},
  'age': {'scales': ['linear', 'log'], 'sourceLink': '', 'use': 'indicator'},
  'age_at_1st_marriage_women': {'name': 'Age at 1st marriage (women)',
   'scales': ['linear'],
   'use': 'indicator'},
  'aged_15_24_employment_rate_percent': {'name': 'Aged 15-24 employment rate (%)',
   'scales': ['linear'],
   'use': 'indicator'},
  'aged_15_24_unemployment_rate_percent': {'name': 'Aged 15-24 unemployment rate (%)',
   'scales': ['linear'],
   'use': 'indicator'},
  'aged_15_64_labour_force_participation_rate_percent': {'name': 'Aged 15-64 labour force participation rate (%)',
   'scales': ['

In [640]:
json.dump(indb_0, open('/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata.json', 'w'), indent=1)

In [109]:
# adding menu level indicators below

In [110]:
en = open('/Users/semio/src/work/Gapminder/vizabi/.data/translation/en.json')
enj = json.load(en)

In [118]:
enj

{'buttons/about': 'About',
 'buttons/axes': 'X and Y',
 'buttons/axesmc': 'X and Y',
 'buttons/axis_x': 'X axis',
 'buttons/axis_y': 'Y axis',
 'buttons/color': 'Color',
 'buttons/colors': 'Color',
 'buttons/deselect': 'Deselect',
 'buttons/desktop': 'Desktop',
 'buttons/expand': 'Expand',
 'buttons/find': 'Find',
 'buttons/label': 'Labels',
 'buttons/lock': 'Lock',
 'buttons/more_options': 'Options',
 'buttons/ok': 'OK',
 'buttons/opacity': 'Opacity',
 'buttons/opacityNonselect': 'Opacity of non-selected',
 'buttons/opacityRegular': 'Regular opacity',
 'buttons/presentation': 'Present',
 'buttons/show': 'Show',
 'buttons/showall': 'Show all',
 'buttons/size': 'Size',
 'buttons/size_label': 'Size',
 'buttons/speed': 'Speed',
 'buttons/stack': 'Stack',
 'buttons/trails': 'Trails',
 'buttons/unexpand': 'Restore',
 'check/adaptMinMaxZoom': 'Follow bubbles with zoom',
 'check/presentation': 'Increase text size for presentation',
 'check/removeLabelBox': 'Remove label boxes',
 'hints/bubbl/

In [114]:
lvl1 = concepts['menu_level1'].dropna().unique()

In [115]:
lvl1

array(['Economy', 'For advanced users', 'Society', 'Education', 'Energy',
       'Environment', 'Health', 'Infrastructure', 'Population', 'Work'], dtype=object)

In [120]:
lvl1_dict = dict([('indicator/'+to_concept_id(x), x) for x in lvl1])

In [121]:
lvl1_dict

{'indicator/economy': 'Economy',
 'indicator/education': 'Education',
 'indicator/energy': 'Energy',
 'indicator/environment': 'Environment',
 'indicator/for_advanced_users': 'For advanced users',
 'indicator/health': 'Health',
 'indicator/infrastructure': 'Infrastructure',
 'indicator/population': 'Population',
 'indicator/society': 'Society',
 'indicator/work': 'Work'}

In [122]:
lvl2 = concepts['menu_level_2'].dropna().unique()

In [123]:
lvl2_dict = dict([('indicator/'+to_concept_id(x), x) for x in lvl2])

In [124]:
lvl2_dict

{'indicator/achievement': 'Achievement',
 'indicator/advanced_debt_trade': 'Advanced debt & trade',
 'indicator/aid_given': 'Aid given',
 'indicator/aid_received': 'Aid received',
 'indicator/alternative_child_mortality_data': 'Alternative child mortality data',
 'indicator/alternative_demography_indicators': 'Alternative demography indicators',
 'indicator/alternative_energy': 'Alternative energy',
 'indicator/alternative_gdp_data': 'Alternative GDP data',
 'indicator/alternative_health_data': 'Alternative health data',
 'indicator/alternative_other_data': 'Alternative other data',
 'indicator/alternative_poverty_measures': 'Alternative poverty measures',
 'indicator/both_sexes_by_age': 'Both sexes by age (%)',
 'indicator/both_sexes_by_age_total': 'Both sexes by age (total)',
 'indicator/cancer_female': 'Cancer, female',
 'indicator/cancer_male': 'Cancer, male',
 'indicator/causes_of_child_deaths': 'Causes of child deaths',
 'indicator/causes_of_newborn_deaths': 'Causes of newborn de

In [127]:
enj['indicator/health']

'Health'

In [133]:
for k in lvl2_dict.keys():
    try:
        print(k, enj[k])
    except:
        continue

In [134]:
enj.update(lvl1_dict)

In [135]:
enj.update(lvl2_dict)

In [140]:
en = open('/Users/semio/src/work/Gapminder/vizabi/.data/translation/en.json', 'w')
json.dump(enj, en, indent=1, sort_keys=True)
en.close()

In [ ]:
# new things
# adding availbity and domain
# remove name


In [242]:
indb_0 = json.load(open('/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata.json'))

In [243]:
indb_0['indicatorsDB']

{'20120905_extreme_poverty_percent_people_below_125_a_day': {'name': 'Extreme poverty (% people below $1.25 a day) version 20120905',
  'scales': ['linear'],
  'use': 'indicator'},
 '_default': {'scales': ['ordinal'], 'sourceLink': '', 'use': 'constant'},
 'adults_with_hiv_percent_age_15_49': {'name': 'Adults with HIV (%, age 15-49)',
  'scales': ['log'],
  'use': 'indicator'},
 'age': {'scales': ['linear', 'log'], 'sourceLink': '', 'use': 'indicator'},
 'age_at_1st_marriage_women': {'name': 'Age at 1st marriage (women)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_24_employment_rate_percent': {'name': 'Aged 15-24 employment rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_24_unemployment_rate_percent': {'name': 'Aged 15-24 unemployment rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_64_labour_force_participation_rate_percent': {'name': 'Aged 15-64 labour force participation rate (%)',
  'scales': ['linear'],
  'use': 'indicator'},
 '

In [244]:
for v in indb_0['indicatorsDB'].values():
    try:
        del v['name']
    except:
        pass

In [228]:
dps = [x for x in os.listdir('./output/') if 'datapoints' in x]

In [245]:
for i in dps:
    tc = re.match(r'ddf--datapoints--(.*)--by--geo--time.csv', i).groups()[0]
    
    df = pd.read_csv('./output/'+i, dtype={tc:float, 'time': int})
    dm = [float(df[tc].min()), float(df[tc].max())]
    av = [int(df['time'].min()), int(df['time'].max())]
    
    if tc in indb_0['indicatorsDB'].keys():
        indb_0['indicatorsDB'][tc].update({'domain':dm, 'availability': av})
    else:
        print(tc)

In [246]:
indb_0['indicatorsDB']

{'20120905_extreme_poverty_percent_people_below_125_a_day': {'availability': [1978,
   2011],
  'domain': [0.0, 92.55],
  'scales': ['linear'],
  'use': 'indicator'},
 '_default': {'scales': ['ordinal'], 'sourceLink': '', 'use': 'constant'},
 'adults_with_hiv_percent_age_15_49': {'availability': [1979, 2011],
  'domain': [0.00951, 26.5],
  'scales': ['log'],
  'use': 'indicator'},
 'age': {'scales': ['linear', 'log'], 'sourceLink': '', 'use': 'indicator'},
 'age_at_1st_marriage_women': {'availability': [1616, 2005],
  'domain': [12.5, 33.66],
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_24_employment_rate_percent': {'availability': [1991, 2007],
  'domain': [11.3, 81.2],
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_24_unemployment_rate_percent': {'availability': [1981, 2005],
  'domain': [2.8, 45.1],
  'scales': ['linear'],
  'use': 'indicator'},
 'aged_15_64_labour_force_participation_rate_percent': {'availability': [1980,
   2007],
  'domain': [39.2, 92.4],
  

In [248]:
json.dump(indb_0, open('/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata.json', 'w'), indent=1)

In [249]:
### add source link

In [263]:
indb_0 = json.load(open('/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata.json'))

In [262]:
urls = cc[['concept', 'indicator_url']]
urls.columns = ['concept', 'sourcelink']

In [264]:
for k, v in urls.set_index('concept').to_dict('index').items():
    if k in indb_0['indicatorsDB'].keys():
        indb_0['indicatorsDB'][k].update(v)
    else:
        print(k)

In [265]:
indb_0['indicatorsDB']

{'20120905_extreme_poverty_percent_people_below_125_a_day': {'availability': [1978,
   2011],
  'domain': [0.0, 92.55],
  'scales': ['linear'],
  'sourcelink': 'https://docs.google.com/spreadsheet/pub?key=0ArfEDsV3bBwCdDhjcXdjbURLMFFVcVFPYThhYmtvZXc',
  'use': 'indicator'},
 '_default': {'scales': ['ordinal'], 'sourceLink': '', 'use': 'constant'},
 'adults_with_hiv_percent_age_15_49': {'availability': [1979, 2011],
  'domain': [0.00951, 26.5],
  'scales': ['log'],
  'sourcelink': 'http://spreadsheets.google.com/pub?key=pyj6tScZqmEfbZyl0qjbiRQ',
  'use': 'indicator'},
 'age': {'scales': ['linear', 'log'], 'sourceLink': '', 'use': 'indicator'},
 'age_at_1st_marriage_women': {'availability': [1616, 2005],
  'domain': [12.5, 33.66],
  'scales': ['linear'],
  'sourcelink': 'http://spreadsheets.google.com/pub?key=t4eF8H_jq_xyKCUHAX6VT1g',
  'use': 'indicator'},
 'aged_15_24_employment_rate_percent': {'availability': [1991, 2007],
  'domain': [11.3, 81.2],
  'scales': ['linear'],
  'sourcelin

In [266]:
json.dump(indb_0, open('/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata.json', 'w'), indent=1)

In [278]:
urls.set_index('concept').ix['aged_65plus_labour_force_participation_rate_percent']

sourcelink    http://spreadsheets.google.com/pub?key=r1hlZB_...
Name: aged_65plus_labour_force_participation_rate_percent, dtype: object

In [279]:
ddf = pd.read_csv('./output/ddf--datapoints--aged_65plus_labour_force_participation_rate_percent--by--geo--time.csv')

In [280]:
ddf.describe()

,aged_65plus_labour_force_participation_rate_percent,time
count,5292.000000,5292.000000
mean,27.059864,1993.500000
std,20.322983,8.078511
min,0.500000,1980.000000
25%,10.000000,1986.750000
50%,22.500000,1993.500000
75%,39.625000,2000.250000
max,89.000000,2007.000000
